In [1]:
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# -*- coding: utf-8 -*-
import pandas as pd, numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ChromeOptions
from datetime import timedelta
import math
import time

Compute articles related to Luzern and save them in a temporary csv file

In [ ]:
# get the date of the moste recent article in the merged dataset
liverss_and_coms_path = pathlib.Path.home() / 'SocialMediaMonitor' / 'data' / 'raw' / 'liverss_and_coms_srf.csv'
liverss_and_coms_df = pd.read_csv(liverss_and_coms_path)
liverss_and_coms_df['published'] = pd.to_datetime(liverss_and_coms_df['published'])
most_recent_article = liverss_and_coms_df['published'].max()

# get the ids of the articles that are less than 15 days old
recent_date = pd.to_datetime(most_recent_article-timedelta(15), utc=True)
mask = (liverss_and_coms_df['published'] > recent_date) # & (total_df['is_com_or_sub'] == 'sub')
recent_articles = liverss_and_coms_df.loc[mask]
recent_articles_id = recent_articles['id'].tolist()
#print(recent_articles_id[:3])
print(len(recent_articles_id))